In [40]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in data
df = pd.read_csv('../../data/wmata.csv', encoding='utf-16', delimiter="\t")

# Remove columns Servicetype_This_Year(group), Holiday_Last_Year, Holiday_This_Year, Servicetype_This_Year, Time_Period, and Date_Last_Year
df = df.drop(['Day_of_Date_This_Year', 'Servicetype_This_Year_(group)', 'Holiday_Last_Year', 'Holiday_This_Year', 'Servicetype_This_Year', 'Time_Period', 'Date_Last_Year', 'Entries_Last_Year'], axis=1)

# Rename columns 
df = df.rename(columns={'Date_This_Year': 'Date'})
df = df.rename(columns={'Entries_This_Year': 'Entries'})

# Pivot data
pivot_df = df.pivot_table(index='Date', columns='Station', values='Entries')

# Convert index of pivot_df to datetime
pivot_df.index = pd.to_datetime(pivot_df.index)

# Organize index of pivot_df from earliest to latest date
pivot_df = pivot_df.sort_index()

# Select stations Anacostia, Stadium-Armory, Van Ness-UDC, Shaw-Howard Univ, Gallery Place, and Capitol South from pivot_df
new_df = pivot_df[['Anacostia', 'Stadium-Armory', 'Van Ness-UDC', 'Shaw-Howard U', 'Gallery Place', 'Capitol South']]


# Save pivot_df and new_df to csv
pivot_df.to_csv('../../data/cleaned_data/wmata_cleaned.csv')
new_df.to_csv('../../data/cleaned_data/wmata_new_cleaned.csv')

########## Index.qmd
# Build interactive timeseries plot using plotly
# Import libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read in wmata_new_cleaned.csv
# new_df = pd.read_csv('../../data/cleaned_data/wmata_new_cleaned.csv')

# List all stations
stations = list(new_df.columns)

# Create subplot with one trace per station
fig = make_subplots(rows=1, cols=1)
for station in stations:
    fig.add_trace(
        go.Scatter(x=new_df.index, y=new_df[station], name=station),
        row=1, col=1
    )

# Create dropdown menu to select station
buttons = []
for station in stations:
    buttons.append(
        dict(method='update', label=station, args=[{'visible': [station == s for s in stations]}])
    )
dropdown = dict(
    active=0, buttons=buttons, direction='down', showactive=True, x=1.1, y=1.1
)

# Update layout
fig.update_layout(
    updatemenus=[dropdown], height=600, width=900,
    title='WMATA Metro Entries by Selected Station and Date in March 2023', xaxis_title='Date', yaxis_title='Entries',
    yaxis=dict(range=[0, 3000])
)

# Show plot
fig.show()


In [41]:
from IPython.display import display, HTML

# Import pivot_df from the file
pivot_df = pd.read_csv('../../data/cleaned_data/wmata_cleaned.csv')
display(HTML(pivot_df.to_html()))


,Date,Addison Road,Anacostia,Archives,Arlington Cemetery,Ashburn,Ballston-MU,Benning Road,Bethesda,Braddock Road,Branch Ave,Brookland-CUA,Capitol Heights,Capitol South,Cheverly,Clarendon,Cleveland Park,College Park-U of Md,Columbia Heights,Congress Heights,Court House,Crystal City,Deanwood,Downtown Largo,Dulles Airport,Dunn Loring,Dupont Circle,East Falls Church,Eastern Market,Eisenhower Ave,Farragut North,Farragut West,Federal Center SW,Federal Triangle,Foggy Bottom-GWU,Forest Glen,Fort Totten,Franconia-Springfield,Friendship Heights,Gallery Place,Georgia Ave-Petworth,Glenmont,Greenbelt,Greensboro,Grosvenor-Strathmore,Herndon,Huntington,Hyattsville Crossing,Innovation Center,Judiciary Square,King St-Old Town,L'Enfant Plaza,Landover,Loudoun Gateway,McLean,McPherson Sq,Medical Center,Metro Center,Minnesota Ave,Morgan Boulevard,Mt Vernon Sq,Navy Yard-Ballpark,Naylor Road,New Carrollton,NoMa-Gallaudet U,North Bethesda,Pentagon,Pentagon City,Potomac Ave,Reston Town Center,Rhode Island Ave,Rockville,Ronald Reagan Washington National Airport,Rosslyn,Shady Grove,Shaw-Howard U,Silver Spring,Smithsonian,Southern Ave,Spring Hill,Stadium-Armory,Suitland,Takoma,Tenleytown-AU,Twinbrook,Tysons,U Street,Union Station,Van Dorn Street,Van Ness-UDC,Vienna,Virginia Sq-GMU,Waterfront,West Falls Church,West Hyattsville,Wheaton,Wiehle-Reston East,Woodley Park
0,2023-03-01,243.6,444.2,540.8,96.50,247.2,1061.0,281.0,913.4,406.8,434.8,684.2,192.2,1023.6,94.4,496.2,433.8,349.6,1267.8,213.8,653.0,992.6,135.2,371.600000,210.4,318.6,1705.6,468.4,627.0,161.60,2183.6,2011.6,458.4,760.2,2359.4,233.4,1099.0,595.4,705.8,2102.6,638.4,618.4,531.4,148.4,447.2,147.40,508.2,440.8,118.6,696.2,557.2,1671.0,149.6,67.00,252.4,1397.8,569.4,2731.0,203.0,158.2,417.4,1024.4,185.8,505.2,1454.2,350.4,1325.8,1055.8,413.8,120.8,636.4,429.8,778.6,1402.6,1049.4,541.2,1060.8,708.2,324.6,103.4,284.2,311.4,584.6,776.0,426.8,298.2,677.8,2399.6,224.0,642.0,747.8,400.2,508.8,215.4,410.8,348.6,327.6,652.4
1,2023-03-02,247.0,469.4,499.8,119.25,208.0,1008.0,275.2,833.6,380.2,406.8,687.6,191.4,936.0,91.2,483.0,403.4,352.6,1204.2,233.0,646.2,1011.4,135.8,338.400000,238.0,294.8,1746.6,457.2,623.2,181.00,2068.4,1879.0,407.6,727.0,2397.6,217.8,1056.8,554.0,684.6,2015.8,660.0,565.8,478.0,171.4,411.4,124.40,513.8,430.8,118.6,648.2,553.0,1411.6,136.2,64.20,237.2,1288.4,556.4,2588.8,204.0,141.6,449.6,967.8,185.6,461.0,1378.0,351.4,1290.2,1016.2,414.0,122.2,600.0,383.2,671.2,1350.2,916.8,493.0,1006.4,686.0,323.2,91.0,290.0,301.6,559.6,722.6,398.6,306.8,666.6,2324.8,217.2,569.6,672.6,396.2,478.8,200.6,388.2,350.4,309.8,647.6
2,2023-03-03,203.6,412.0,474.8,84.50,152.2,815.6,243.2,639.6,266.2,315.8,608.4,171.4,746.6,65.4,391.0,352.6,333.6,1139.6,198.4,513.2,755.0,123.0,278.000000,241.0,223.2,1434.2,339.2,550.4,123.20,1368.2,1235.6,316.2,561.8,1977.8,149.8,909.4,451.8,542.6,1929.2,587.4,437.6,376.8,96.4,320.8,98.20,434.0,397.4,91.2,455.2,436.6,1201.2,110.0,35.60,142.6,879.6,454.6,1795.8,176.0,119.2,463.4,823.6,139.8,361.0,1065.4,255.8,1055.2,901.6,338.0,81.2,540.8,308.8,696.8,1033.6,740.4,500.4,840.8,646.2,332.4,79.0,240.4,265.2,453.0,617.4,339.8,263.6,679.2,1856.2,178.8,458.4,528.8,297.6,438.0,134.8,352.2,304.2,251.6,491.6
3,2023-03-04,199.2,207.4,390.8,219.25,146.6,753.0,132.2,502.2,210.0,145.8,381.8,84.2,342.6,27.8,450.8,337.6,385.6,1019.0,129.2,435.2,599.4,63.0,15.800000,226.2,188.2,1222.4,278.4,574.6,99.40,661.0,571.2,221.0,327.0,1576.4,98.8,529.0,327.6,379.2,1933.8,419.2,278.6,306.8,83.8,221.4,83.60,254.6,291.2,90.4,225.6,503.0,790.6,49.2,17.20,102.2,609.6,110.4,1554.0,87.4,15.0,389.6,679.2,69.4,230.4,988.6,188.2,152.0,916.8,238.2,69.6,421.8,220.2,581.2,805.6,513.2,342.8,680.8,1023.0,144.4,56.2,156.4,105.6,295.2,481.6,239.2,331.4,709.6,1050.2,125.4,337.8,420.2,283.6,419.2,97.8,222.2,241.4,192.2,588.4
4,2023-03-05,165.0,168.2,273.6,236.75,147.8,469.8,98.6,381.0,173.0,133.6,273.0,58.6,173.4,26.4,302.2,234.0,236.0,672.0,117.6,329.6,427.4,52.6,3.666667,273.8,120.4,967.0,229.4,448.0,76.80,468.6,425.2